In [ ]:
import asyncpraw
import nest_asyncio
import csv
import asyncio

nest_asyncio.apply()

async def get_comments(submission, topics, retry_count=0):
    """get comments with retry logic."""
    try:
        await submission.comments.replace_more(limit=None)
        return [
            {
                'subreddit': comment.subreddit.display_name,
                'commentId': comment.id,
                'commentTimeStamp': comment.created_utc,
                'commentAuthor': comment.author,
                'comment': comment.body
            } # only return those comments that have any keywords from the topics lists
            for comment in submission.comments.list() if any(keyword in comment.body for keyword in topics)
        ]
    except asyncpraw.exceptions.RedditAPIException as e:
        print(f"Reddit API exception: {e}")
        if retry_count < 3:  # Retry up to 3 times
            await asyncio.sleep(10 * retry_count)  # Exponential backoff
            return await get_comments(submission, topics, retry_count + 1)
        else:
            raise

async def add_to_csv(entry, filename):
    """Append a single entry to the CSV file."""
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        fieldnames = ['subreddit', 'commentId', 'commentTimeStamp', 'commentAuthor', 'comment', 'topic']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        for comment in entry['comments']:
            comment['topic'] = entry['topic']
            writer.writerow(comment)

async def scrape_reddit_comments(topics, filename, limit=1000):
    reddit = asyncpraw.Reddit(
        client_id="your_client_id",
        client_secret="your_client_secret",
        user_agent="your_user_agent"
    )
    
    # Initialize CSV file with headers
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ['subreddit', 'commentId', 'commentTimeStamp', 'commentAuthor', 'comment', 'topic']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

    for topic in topics:
        all_posts = await reddit.subreddit('all') #search all subreddit with these topics
        top_posts = all_posts.search(topic, limit=limit)

        async for post in top_posts:
            try:
                submission = await reddit.submission(id=post.id)
                comments = await get_comments(submission, topics)
                entry = {
                    "topic": topic,
                    "comments": comments
                }
                await add_to_csv(entry, filename)
            except asyncpraw.exceptions.RedditAPIException:
                await asyncio.sleep(10)
                continue
            except Exception as e:  # Specific exceptions are preferred
                print(f"An error occurred: {e}")
                await asyncio.sleep(10)
                continue

    await reddit.close()

# List of topics to search
topics = [
    'russia',
    'ukraine Conflict',
    '.russia.ukraine.',
    'ukraine',
    'Russian.war.',
    'russia.invades.Ukraine',
    'ukraine.war.',
    'UA',
    'RU',
    'Ukrainian',
    'Israel',
    'Palestine',
    'israeli',
    'israel'
    'gaza',
    'palestine'
    'hamas',
]

# Set the filename
filename = 'ru_ip_war_comment.csv'

# Run the script and save the data for each topic
await scrape_reddit_comments(topics, filename, limit=1000)

print("Data saved to 'ru_ip_war_comment'")
